In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlalchemy import create_engine

import os
import csv
from datetime import datetime

In [2]:
from sklearn.externals import joblib ## model persistance

In [71]:
pkloc = '/home/master/Data/dummy/'
reploc = pkloc
loc = ''
db_name = 'random_train.db'

###  # create report.csv file with header (Time_taken) * rest adds auto

In [146]:
def save_report(msg,di):               # create csv file with header
    rep = "\n"+msg+" :\n\t"+str(di)
    fp = open(reploc+'report.txt','a')
    fp.write(rep)
    fp.close()

def save_report_csv(msg,di):               # create csv file with header
    cn = pd.read_csv(reploc+'report.csv')
    for k,v in di.items():
        cn.loc[msg,k] = v
    #cn.to_csv(reploc+'report.csv')
    return cn
    

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(loc+db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
 
    return None

In [5]:
con = create_connection(db_name)

In [19]:
tag_rm = list( pd.read_csv('tag_to_remove.csv')['Tags'] )
t = [str(i) for i in tag_rm]
li_string = "('"+ "','".join(t) +"')" # li_string = '("'+ '","'.join(t) +'")' #  ' vs "

In [20]:
if con is not None:
    tag_data = pd.read_sql('SELECT Tags FROM data WHERE Tags NOT IN '+li_string,con)
else :
    print('Conn error')

In [21]:
print(tag_data.head())
print('Shape',tag_data.shape)

                                                Tags
0             iphone objective-c ios uiview uibutton
1                    svg internet-explorer-9 raphael
2  validation spring-mvc internationalization cus...
3                             windows java copy text
4                                  javascript jquery
Shape (9708, 1)


In [208]:
if con is not None:
    t_data = pd.read_sql('SELECT Title FROM data WHERE Tags NOT IN '+li_string,con)
else :
    print('Conn error')


In [209]:
print(t_data.head())
print('Shape',t_data.shape)

                                               Title
0       One tap triggering events on multiple views?
1  IE9 text positioning bug when zoomed in with R...
2  Spring MVC custom errors and internationalization
3                How to copy text from Java program?
4  How to scroll to a part of the page using jQuery?
Shape (9708, 1)


## Preprocess title Data

In [210]:
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")
# https://stackoverflow.com/questions/35345761/python-re-split-vs-nltk-word-tokenize-and-sent-tokenize

In [211]:
import re

#t_data.head()
#t_data.Title = t_data.Title.apply(lambda x : x.encode('utf-8'))
t_data.Title = t_data.Title.apply(lambda x : str.lower(str(x)))
t_data.Title = t_data.Title.apply(lambda x : re.sub(r'[^A-Za-z0-9#+.\-]+',' ',x))
#title_data = t_data


In [212]:
def fn(sen):
    return ' '.join(str(stemmer.stem(w)) for w in word_tokenize(sen) if w not in stop_words and (len(w)!=1 or w=='c'))

t_data.Title = t_data.Title.apply(lambda x : fn(x))#' '.join for w in word_tokenize(x) if w not in stop_words)


In [215]:
t_data.Title[:5]

0     one tap trigger event multipl view
1    ie9 text posit bug zoom raphael svg
2    spring mvc custom error internation
3                 copi text java program
4            scroll part page use jqueri
Name: Title, dtype: object

In [216]:
title_data = t_data

### ----- Body

In [175]:
if con is not None:
    b_data = pd.read_sql('SELECT Body FROM data  WHERE Tags NOT IN '+li_string,con)
else :
    print('Conn error')

In [176]:
print(b_data[:5])
print('Shape',b_data.shape)

                                                Body
0  <p>In my program I have placed a <code>UIButto...
1  <p>I'm not sure if this is a bug with Raphael ...
2  <p>In my web application, I handle errors with...
3  <p>I am using a Java program called <a href="h...
4  <p>My code scrolls the user to the bottom of t...
Shape (9708, 1)


In [177]:
b_data.Body = b_data.Body.apply(lambda x : x.encode('utf-8'))
b_data.Body = b_data.Body.apply(lambda x : str.lower(str(x)))
b_data.Body = b_data.Body.apply(lambda x : re.sub(r'<code>(.*?)</code>',' ',x))
b_data.Body = b_data.Body.apply(lambda x : re.sub(r'<.*?>',' ',x))
b_data.Body = b_data.Body.apply(lambda x : re.sub(r'[^A-Za-z0-9#+.\-]+',' ',x))

b_data.Body = b_data.Body.apply(lambda x : fn(x))

In [178]:
b_data[:5]

Body
0  program place subview set button tap event cal...
1  sure bug raphael svg anyon fix workaround woul...
2  web applic handl error annot everyth work fine...
3  use java program call jdownload v0.9.581 wish ...
4  code scroll user bottom page modifi take user ...

In [179]:
title_data = b_data

## ---------------- title+body = ques

In [38]:
b_data.shape

(9805, 1)

In [39]:
t_data.shape

(9805, 1)

In [40]:
title_data = pd.DataFrame()
title_data['ques'] = t_data.Title + b_data.Body
title_data.shape

(9805, 1)

### --------------------------------------

In [202]:
from sklearn.feature_extraction.text import CountVectorizer

In [203]:
vectorizer_tag = CountVectorizer(tokenizer = lambda x: x.split(), binary='true',min_df = 5)  # min_df - document frequency strictly lower than the given threshold

In [204]:
multi_lab_y = vectorizer_tag.fit_transform(tag_data.Tags)

In [205]:
vectorizer_tag

CountVectorizer(analyzer='word', binary='true', decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function <lambda> at 0x7fea70553ae8>, vocabulary=None)

In [207]:
joblib.dump(tag_data.Tags,pkloc+'tag_data.pkl')

['/home/master/Data/dummy/tag_data.pkl']

In [48]:
print("Number of data points :", multi_lab_y.shape[0])
print("Number of unique tags :", multi_lab_y.shape[1])

Number of data points : 9708
Number of unique tags : 939


In [144]:
#vectorizer_tag.get_feature_names()

In [145]:
## choose first n tags (desc order of count)

def tags_to_choose(n):
    t = multilabel_y.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(t)), key=lambda i: t[i], reverse=True)
    multilabel_yn = multilabel_y[:,sorted_tags_i[:n]]
    return multilabel_yn

<h2>4.2 Split the data into test and train (80:20) </h2>

In [217]:
tot_size = title_data.shape[0]
train_size = int(tot_size * 0.8)
print(tot_size,train_size)

9708 7766


In [218]:
x_train=title_data.head(train_size)
x_test=title_data.tail(tot_size - train_size)

y_train = multi_lab_y[0:train_size,:]
y_test = multi_lab_y[train_size:tot_size,:]

In [200]:
# joblib.dump(y_test,pkloc+'y_test.pkl')

['/home/master/Data/dummy/y_test.pkl']

## @ Model Testing

In [219]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC

from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score

In [220]:
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=200000, smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))

In [221]:
start = datetime.now()
x_train_multilabel = vectorizer.fit_transform(x_train.Title)     ## replace column name appropiately
x_test_multilabel = vectorizer.transform(x_test.Title)
print("Time taken to run this cell :", datetime.now() - start)

Time taken to run this cell : 0:00:00.420396


In [222]:
print("Dimensions of train data X:",x_train_multilabel.shape, "Y :",y_train.shape)
print("Dimensions of test data X:",x_test_multilabel.shape,"Y:",y_test.shape)

Dimensions of train data X: (7766, 64691) Y : (7766, 939)
Dimensions of test data X: (1942, 64691) Y: (1942, 939)


In [187]:
joblib.dump(x_test_multilabel,pkloc+'body_xtest_multilabel.pkl')

['/home/master/Data/dummy/body_xtest_multilabel.pkl']

### SGDC with OnevsRest

In [223]:
start = datetime.now()

classifier = OneVsRestClassifier(SGDClassifier(loss='hinge', alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel, y_train)
predictions = classifier.predict(x_test_multilabel)

time_t = datetime.now() - start
#print("macro f1 score :",metrics.f1_score(y_test, predictions, average = 'macro'))
#print("micro f1 scoore :",metrics.f1_score(y_test, predictions, average = 'micro'))
#print("hamming loss :",metrics.hamming_loss(y_test,predictions))
#print("Precision recall report :\n",metrics.classification_report(y_test, predictions))


In [224]:
rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions, average = 'micro')
print(str(rep_di))

{'Time_taken': '0:00:33.137024', 'accuracy': 0.12203913491246138, 'micro_f1_scoore': 0.4145579506404249}


In [225]:
save_report("SGDC with OnevsRest - hinge",rep_di)

In [226]:
joblib.dump(classifier,pkloc+'title_sgdc_hinge.pkl')

['/home/master/Data/dummy/title_sgdc_hinge.pkl']

### Logistic with OnevsRest

In [192]:
start = datetime.now()

classifier2 = OneVsRestClassifier(LogisticRegression(penalty='l1'), n_jobs=-1)
classifier2.fit(x_train_multilabel, y_train)
predictions2 = classifier2.predict(x_test_multilabel)

time_t = datetime.now() - start

In [193]:

rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions2)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions2, average = 'micro')
print(str(rep_di))

{'Time_taken': '0:03:08.959775', 'accuracy': 0.08496395468589084, 'micro_f1_scoore': 0.2856091277143909}


In [194]:
save_report("Logistic with OnevsRest",rep_di)

In [195]:
joblib.dump(classifier2,pkloc+'body_logistic.pkl')

['/home/master/Data/dummy/body_logistic.pkl']

## LDA  -- TIME

In [204]:
y_train3 =y_train.todense()
y_test3 = y_test.todense()

print type(y_train)
print type(x_train_multilabel.todense())

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.matrixlib.defmatrix.matrix'>


In [ ]:
start = datetime.now()

#classifier3 = OneVsRestClassifier(LDA())
#classifier3.fit(x_train_multilabel.todense(), y_train)
#predictions3 = classifier3.predict(x_test_multilabel)

#print("Time taken to run this cell :", datetime.now() - start)
#print("accuracy :",metrics.accuracy_score(y_test,predictions3))

## SVC

In [152]:
start = datetime.now()

classifier4 = OneVsRestClassifier(LinearSVC())
classifier4.fit(x_train_multilabel, y_train)
predictions4 = classifier4.predict(x_test_multilabel)

time_t = datetime.now() - start


In [153]:

rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions2)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions2, average = 'micro')
print(str(rep_di))

{'Time_taken': '0:00:16.396039', 'accuracy': 0.10710607621009269, 'micro_f1_scoore': 0.3978883861236802}


In [154]:
save_report("SVC",rep_di)

In [155]:
joblib.dump(classifier4,pkloc+'title_svc.pkl')

['/home/master/Data/dummy/title_svc.pkl']

In [166]:
from skmultilearn.adapt import MLkNN
classifier_k = MLkNN(k=21)

In [167]:
classifier_k.fit(x_train_multilabel, y_train)

MLkNN(ignore_first_neighbours=0, k=21, s=1.0)

In [168]:
predictions5 = classifier_k.predict(x_test_multilabel)

In [169]:
rep_di = dict()
rep_di["Time_taken"] = str(time_t)
rep_di["accuracy"] = metrics.accuracy_score(y_test,predictions5)
rep_di["micro_f1_scoore"] = metrics.f1_score(y_test, predictions5, average = 'micro')
print(str(rep_di))

{'Time_taken': '0:00:16.396039', 'accuracy': 0.08650875386199794, 'micro_f1_scoore': 0.25004876145894284}


In [170]:
save_report("MLkNN 21",rep_di)

In [62]:
type(y_test)

scipy.sparse.csr.csr_matrix

In [178]:
actual = vectorizer_tag.inverse_transform(y_test)
actual = [' '.join(i) for i in actual]
actual = pd.Series(actual,name="Actual")
#actual

In [53]:
preds = vectorizer_tag.inverse_transform(predictions)
preds = [' '.join(i) for i in preds]
preds = pd.Series(preds,name="Pred_by_ques")
#preds

In [190]:
resul_df = pd.concat([actual,preds],axis=1)
#resul_df.to_csv("01_resul_rep.csv",index=None)

In [191]:
#rr = pd.read_csv('01_resul_rep.csv')

In [158]:
#rr.append?

In [213]:
rr['pred_by_ques']=preds

In [215]:
rr.to_csv("01_rep_mindf_03.csv",index=None)

In [59]:
r = pd.read_csv('01_rep_mindf_03.csv')

In [178]:
#r

## load and run

In [171]:
title_x_test_multilabel = joblib.load('/data/title_xtest_multilabel.pkl')

title_sgc = joblib.load('/data/title_sgdc.pkl')

pred_title = title_sgc.predict(title_x_test_multilabel)

print("From title accuracy :",metrics.accuracy_score(y_test,pred_title))
print("From title micro f1 scoore :",metrics.f1_score(y_test, pred_title, average = 'micro'))

('From title accuracy :', 0.092809790922998472)
('From title micro f1 scoore :', 0.37282442748091604)


In [172]:
body_x_test_multilabel = joblib.load('/data/body_xtest_multilabel.pkl')

body_sgc = joblib.load('/data/body_sgdc.pkl')

pred_body = body_sgc.predict(body_x_test_multilabel)

print("From body accuracy :",metrics.accuracy_score(y_test,pred_body))
print("From body micro f1 scoore :",metrics.f1_score(y_test, pred_body, average = 'micro'))

('From body accuracy :', 0.079551249362570117)
('From body micro f1 scoore :', 0.31029435680598472)


In [167]:
p_or = pred_title.todense() | pred_body.todense()

In [173]:
print("From OR accuracy :",metrics.accuracy_score(y_test,p_or))
print("From OR micro f1 scoore :",metrics.f1_score(y_test, p_or, average = 'micro'))

('From OR accuracy :', 0.10912799592044875)
('From OR micro f1 scoore :', 0.44913991602329678)


In [176]:
preds = vectorizer_tag.inverse_transform(p_or)
preds = [' '.join(i) for i in preds]
preds = pd.Series(preds,name="Pred_by_OR")
r['pred_by_OR'] = preds
r.to_csv("01_rep_mindf_03.csv",index=None)

In [177]:
r = pd.read_csv('01_rep_mindf_03.csv')
r

Actual  \
0                        response list web-services   
1                              json-encode ajax php   
2                                 upgrade windows-7   
3                                     prime-numbers   
4                            django database python   
5                                         newline c   
6                                navigation android   
7                              android web-services   
8                conflict version-control merge git   
9         cocos2d-iphone cocos2d objective-c iphone   
10                 closures compilation google text   
11                   visual-studio-2012 c++ windows   
12                                           jquery   
13                         utf-8 encoding date java   
14                            cross-browser asp.net   
15                  asp.net web-services javascript   
16                                       jsf jquery   
17                   mod-wsgi django apache2 python   
18                           android-intent android   
19                                  binary file c++   
20       sharepoint-designer compression sharepoint   
21                                      haml jquery   
22                                 ipad objective-c   
23                                        adobe-cs5   
24                       mvvm castle-windsor wpf c#   
25                                              wcf   
26                                               c#   
27                                       blackberry   
28                                          android   
29                                  sql-server-2008   
...                                             ...   
1931                                mediaelement.js   
1932  cursor jpa-2.0 tsql java-ee stored-procedures   
1933                                           solr   
1934            forms ruby-on-rails-3 ruby-on-rails   
1935                     css3 webkit javascript ios   
1936                                        url ssl   
1937                                     ruby regex   
1938                                  download wget   
1939                         iframe html javascript   
1940                          live-streaming ffmpeg   
1941                       colors message text java   
1942                    knockout.js html javascript   
1943                      uiwebview ios objective-c   
1944                                   sorting ruby   
1945                           ruby-on-rails jquery   
1946                                 vb.net asp.net   
1947                                 maven jboss7.x   
1948                                oop winforms c#   
1949        android-listview android-button android   
1950                  coffeescript inline iteration   
1951                                           java   
1952                     solr hosting search apache   
1953                             web service iphone   
1954                        charts flash javascript   
1955                                        xml c++   
1956                                        xml php   
1957                                         oracle   
1958                                 plot vba excel   
1959                                      excel xml   
1960                 python-3.x random regex python   

                            Pred_by_Title                  pred_by_body  \
0                            web-services                  web-services   
1                                     php                      ajax php   
2                               windows-7             windows windows-7   
3                                     NaN                           NaN   
4                           django python                        django   
5                                     NaN                           NaN   
6                                     NaN                       android   
7                                 